In [1]:
import pyodbc
import pandas.io.sql
import xlrd
import pandas as pd
from sqlalchemy import create_engine
import os
import configparser
import readconfig
from configparser import ConfigParser

# Part 1

In [2]:
# Check Driver name

driver_name = ''
driver_names = [x for x in pyodbc.drivers() if x.endswith(' for SQL Server')]
driver_names

['ODBC Driver 17 for SQL Server']

In [3]:
# Read data

data = pd.read_excel('data_source.xlsx', sheet_name=None)

In [10]:
# Connect to SQL

engine = create_engine('mssql+pyodbc://@LAPTOP-T3G74HK4/Orders_RELATION_DB?driver=ODBC+Driver+17+for+SQL+Server')
engine.connect()

In [24]:

os.chdir("C:/Users/PREDATOR/Desktop/AUA/Business Intelligence/Group Project 2")
 

conn_ER = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                         'Server=LAPTOP-T3G74HK4;'
                         'Database=Orders_RELATION_DB;'
                         'Trusted_Connection=yes;')
conn_DW = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                         'Server=LAPTOP-T3G74HK4;'
                         'Database=Orders_DIMENSIONAL_DB;'
                         'Trusted_Connection=yes;')

# Create a Cursor class instance for executing T-SQL statements
cursor_ER = conn_ER.cursor()
cursor_DW = conn_DW.cursor()

In [11]:
# Populate tables

data["OrderDetails"].to_sql('OrderDetails', con = engine, if_exists='replace', index = False)
data["Orders"].to_sql('Orders', con = engine, if_exists='replace', index = False)
data["Products"].to_sql('Products', con = engine, if_exists='replace', index = False)
data["Shippers"].to_sql('Shippers', con = engine, if_exists='replace', index = False)
data["Customers"].to_sql('Customers', con = engine, if_exists='replace', index = False)
data["Suppliers"].to_sql('Suppliers', con = engine, if_exists='replace', index = False)
data["Categories"].to_sql('Categories', con = engine, if_exists='replace', index = False)
data["Customers"].to_sql('Customers', con = engine, if_exists='replace', index = False)
data["EmployeeTerritories"].to_sql('EmployeeTerritories', con = engine, if_exists='replace', index = False)
data["Employees"].to_sql('Employees', con = engine, if_exists='replace', index = False)
data["Territories"].to_sql('Territories', con = engine, if_exists='replace', index = False)
data["Region"].to_sql('Region', con = engine, if_exists='replace', index = False)

# Part 2

In [40]:
# engine2 = create_engine('mssql+pyodbc://@LAPTOP-T3G74HK4/Orders_DIMENSIONAL_DB?driver=ODBC+Driver+17+for+SQL+Server')
# connection2 = engine2.raw_connection()
# cursor2 = connection2.cursor()

In [13]:
# Extract the tables names of the database (excluding system tables)    
def extract_tables_db(cursor, *args):
    results = []
    for x in cursor.execute('exec sp_tables'):
        if x[1] not in args:
            results.append(x[2])
    return results

# Extract the column names of a table given it's name
def extract_table_cols(cursor, table_name):
    result = []
    for row in cursor.columns(table=table_name):
        result.append(row.column_name)
    return result

# A function for finding the primary key of a table
def find_primary_key(cursor, table_name, schema):
    
    # Find the primary key information
    table_primary_key =  cursor.primaryKeys(table_name, schema=schema)
    
    # Store the info about the PK constraint into a dictionary
    columns = [column[0] for column in cursor.description]
    results = []
    for row in cursor.fetchall():
        results.append(dict(zip(columns, row)))
    try:
        return results[0]
    except:
        pass
    return results

In [29]:
def populate_dim_scd1(cursor_src,cursor_dst, src_db, src_table, dst_db, dst_table,connect_col,
                      src_schema = 'dbo', dst_schema = 'dbo',):
    
    # Set the proc name
    proc_name = dst_db + '_'+ dst_table + '_SCD_ETL1'

    # Create column lists
    collist_insert = extract_table_cols(cursor_src, src_table)
    collist_src = ["SRC." + i for i in extract_table_cols(cursor_src, src_table)]
    collist_dst = ["DST." + i for i in extract_table_cols(cursor_dst, dst_table)][1:]

    # Create a check string list
    check_list = []
    for i,j  in zip(collist_src, collist_dst):
        check_list.append(f"Isnull({j}, '') <> Isnull({i}, '')")
    check_str = " OR ".join(check_list)

    # Create a set string list
    set_list = []
    for i,j  in zip(collist_src, collist_dst):
        set_list.append(f"{j} = {i}")
    set_str = ",".join(set_list)

    sql_script_drop_proc = f"""
    DROP PROCEDURE IF EXISTS {proc_name};
    """
    sql_script_create_proc = f"""
    CREATE PROCEDURE {proc_name}
    AS
    BEGIN TRY
    MERGE {dst_db}.{dst_schema}.{dst_table} AS DST 
    USING {src_db}.{src_schema}.{src_table} AS SRC
    ON ( SRC.{connect_col} = DST.{connect_col})
    WHEN NOT MATCHED THEN 
    INSERT ({",".join(collist_insert)})
    VALUES ({",".join(collist_src)})
    WHEN MATCHED AND (
        {check_str}
        ) 
    THEN
        UPDATE 
        SET 
            {set_str}; 
    END TRY
    BEGIN CATCH
        THROW
    END CATCH
    """

    # Dropping the proc if it exists and commiting the change
    cursor_dst.execute(sql_script_drop_proc)
    cursor_dst.commit()

    # Creating a new procedure and committing the change
    cursor_dst.execute(sql_script_create_proc)
    cursor_dst.commit()

    # Executing the created procedure
    cursor_dst.execute("exec " + dst_db + '.' + dst_schema + '.' + proc_name + ';')
    cursor_dst.commit()

    return f'The {dst_table} table of the {dst_db} database has been populated!'

In [49]:
extract_table_cols(cursor_ER, 'Employees')

['EmployeeID',
 'LastName',
 'FirstName',
 'Title',
 'TitleOfCourtesy',
 'BirthDate',
 'HireDate',
 'Address',
 'City',
 'Region',
 'PostalCode',
 'Country',
 'HomePhone',
 'Extension',
 'Notes',
 'ReportsTo',
 'PhotoPath']

In [50]:
def populate_dim_scd2(cursor_src,cursor_dst, src_db, src_table, dst_db, dst_table,connect_col,
                      src_schema = 'dbo', dst_schema = 'dbo',):
    
    # Set the proc name
    proc_name = dst_db + '_'+ dst_table + '_SCD_ETL2'

    # Create column lists
    collist_insert = extract_table_cols(cursor_src, src_table)
    collist_insert2 = collist_insert
    collist_src = ["SRC." + i for i in extract_table_cols(cursor_src, src_table)]
    collist_srcEXTRA = collist_src
    collist_srcEXTRA.append('@Today')
    collist_srcEXTRA.append('1')
    collist_srcEXTRA2 = collist_src
    collist_srcEXTRA2.append('ValidFrom')
    collist_srcEXTRA2.append('IsCurrent')
    collist_dst = ["DST." + i for i in extract_table_cols(cursor_dst, dst_table)][1:]

    # Create a check string list
    check_list = []
    for i,j  in zip(collist_src, collist_dst):
        check_list.append(f"Isnull({j}, '') <> Isnull({i}, '')")
    check_str = " OR ".join(check_list)

    # Create a set string list
    set_list = []
    for i,j  in zip(collist_src, collist_dst):
        set_list.append(f"{j} = {i}")
    set_str = ",".join(set_list)

    sql_script_drop_proc = f"""
    DROP PROCEDURE IF EXISTS {proc_name};
    """
    sql_script_create_proc = f"""
    CREATE PROCEDURE {proc_name}
    AS
    BEGIN TRY
    DECLARE @Yesterday INT = (YEAR(DATEADD(dd,-1,GETDATE())) * 10000) + (MONTH(DATEADD(dd,-1,GETDATE())) * 100) + DAY(DATEADD(dd,-1,GETDATE()))
    DECLARE @Today INT = (YEAR(GETDATE()) * 10000) + (MONTH(GETDATE()) * 100) + DAY(GETDATE())
    INSERT INTO {dst_db}.{dst_schema}.{dst_table}({",".join(collist_insert)})
    SELECT {",".join(collist_insert)}
    FROM
    (
    MERGE INTO {dst_db}.{dst_schema}.{dst_table} AS DST 
    USING {src_db}.{src_schema}.{src_table} AS SRC
    ON ( SRC.{connect_col} = DST.{connect_col})
    WHEN NOT MATCHED THEN
    INSERT ({",".join(collist_srcEXTRA2)})
    VALUES ({",".join(collist_srcEXTRA)})
    WHEN MATCHED
    AND IsCurrent = 1
    AND (
     ISNULL(DST.LastName,'') <> ISNULL(SRC.LastName,'') 
     OR ISNULL(DST.FirstName,'') <> ISNULL(SRC.FirstName,'') 
     OR ISNULL(DST.Title,'') <> ISNULL(SRC.Title,'')
     OR ISNULL(DST.TitleOfCourtesy,'') <> ISNULL(SRC.TitleOfCourtesy,'')
     OR ISNULL(DST.BirthDate,'') <> ISNULL(SRC.BirthDate,'')
     OR ISNULL(DST.HireDate,'') <> ISNULL(SRC.HireDate,'')
     OR ISNULL(DST.Address,'') <> ISNULL(SRC.Address,'')
     OR ISNULL(DST.City,'') <> ISNULL(SRC.City,'')
     OR ISNULL(DST.Region,'') <> ISNULL(SRC.Region,'')
     OR ISNULL(DST.PostalCode,'') <> ISNULL(SRC.PostalCode,'')
     OR ISNULL(DST.Country,'') <> ISNULL(SRC.Country,'')
     OR ISNULL(DST.HomePhone,'') <> ISNULL(SRC.HomePhone,'')
     OR ISNULL(DST.Extension,'') <> ISNULL(SRC.Extension,'')
     OR ISNULL(DST.Notes,'') <> ISNULL(SRC.Notes,'')
     OR ISNULL(DST.ReportsTo,'') <> ISNULL(SRC.ReportsTo,'')
     OR ISNULL(DST.PhotoPath,'') <> ISNULL(SRC.PhotoPath,'')
     )
    THEN UPDATE 
    SET DST.IsCurrent = 0, DST.ValidTo = @Yesterday
    OUTPUT SRC.EmployeeID, SRC.LastName, SRC.FirstName, SRC.Title, SRC.TitleOfCourtesy, SRC.BirthDate, SRC.HireDate, SRC.Address, SRC.City, SRC.Region, SRC.PostalCode, SRC.Country, SRC.HomePhone, SRC.Extension, SRC.Photo, SRC.Notes, SRC.ReportsTo, SRC.PhotoPath, $Action AS MergeAction
    ) AS MRG
    WHERE MRG.MergeAction = 'UPDATE'
    ;
    END TRY
    BEGIN CATCH
        THROW
    END CATCH
    """

    # Dropping the proc if it exists and commiting the change
    cursor_dst.execute(sql_script_drop_proc)
    cursor_dst.commit()

    # Creating a new procedure and committing the change
    cursor_dst.execute(sql_script_create_proc)
    cursor_dst.commit()

    # Executing the created procedure
    cursor_dst.execute("exec " + dst_db + '.' + dst_schema + '.' + proc_name + ';')
    cursor_dst.commit()

    return f'The {dst_table} table of the {dst_db} database has been populated!'

In [31]:
populate_dim_scd1(cursor_ER, cursor_DW, src_db = 'Orders_RELATION_DB', src_table = 'Shippers',
              dst_db = 'Orders_DIMENSIONAL_DB', dst_table = 'DimShippers',
              connect_col = 'ShipperID')

'The DimShippers table of the Orders_DIMENSIONAL_DB database has been populated!'

In [ ]:
populate_dim_scd2(cursor_ER, cursor_DW, src_db = 'Orders_RELATION_DB', src_table = 'Employees',
              dst_db = 'Orders_DIMENSIONAL_DB', dst_table = 'DimEmployee',
              connect_col = 'EmployeeID')

In [32]:
for row in cursor_DW.execute('SELECT * FROM DimShippers'):
    print(row)